<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2025-02-15 12:39:10.229638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-15 12:39:10.445260: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 12:39:10.453011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-15 12:39:10.453045: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2025-02-15 12:39:16.902719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-15 12:39:16.902770: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-15 12:39:16.902799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-sadeghkhajep): /proc/driver/nvidia/version does not exist
2025-02-15 12:39:16.903126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 39s 48ms/step - loss: 0.6736 - accuracy: 0.5882 - val_loss: 0.6301 - val_accuracy: 0.6528
Epoch 2/10
782/782 [==============================] - 38s 48ms/step - loss: 0.5807 - accuracy: 0.6949 - val_loss: 0.5611 - val_accuracy: 0.7066
Epoch 3/10
782/782 [==============================] - 36s 46ms/step - loss: 0.5107 - accuracy: 0.7490 - val_loss: 0.5187 - val_accuracy: 0.7368
Epoch 4/10
782/782 [==============================] - 37s 48ms/step - loss: 0.4657 - accuracy: 0.7786 - val_loss: 0.4982 - val_accuracy: 0.7537
Epoch 5/10
782/782 [==============================] - 36s 46ms/step - loss: 0.4345 - accuracy: 0.7990 - val_loss: 0.4786 - val_accuracy: 0.7648
Epoch 6/10
782/782 [==============================] - 38s 48ms/step - loss: 0.4118 - accuracy: 0.8133 - val_loss: 0.4686 - val_accuracy: 0.7710
Epoch 7/10
782/782 [==============================] - 36s 45ms/step - loss: 0.3952 - accuracy: 0.8220 - val_loss: 0.4587 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 8s 10ms/step - loss: 0.4544 - accuracy: 0.7831
Test score: 0.45440515875816345
Test accuracy: 0.7831199765205383


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 81s 102ms/step - loss: 0.6563 - accuracy: 0.6098 - val_loss: 0.5932 - val_accuracy: 0.6968
Epoch 2/10
782/782 [==============================] - 81s 104ms/step - loss: 0.5258 - accuracy: 0.7458 - val_loss: 0.5035 - val_accuracy: 0.7590
Epoch 3/10
782/782 [==============================] - 77s 99ms/step - loss: 0.4451 - accuracy: 0.7974 - val_loss: 0.4624 - val_accuracy: 0.7804
Epoch 4/10
782/782 [==============================] - 78s 100ms/step - loss: 0.3932 - accuracy: 0.8284 - val_loss: 0.4492 - val_accuracy: 0.7888
Epoch 5/10
782/782 [==============================] - 78s 100ms/step - loss: 0.3593 - accuracy: 0.8446 - val_loss: 0.4252 - val_accuracy: 0.8032
Epoch 6/10
782/782 [==============================] - 81s 104ms/step - loss: 0.3354 - accuracy: 0.8560 - val_loss: 0.4109 - val_accuracy: 0.8133
Epoch 7/10
782/782 [==============================] - 79s 101ms/step - loss: 0.3175 - accuracy: 0.8674 - val_loss: 0.4067 - val_acc

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 71s 89ms/step - loss: 0.6919 - accuracy: 0.5091 - val_loss: 0.6871 - val_accuracy: 0.5019
Epoch 2/10
782/782 [==============================] - 69s 89ms/step - loss: 0.6538 - accuracy: 0.5880 - val_loss: 0.5919 - val_accuracy: 0.7030
Epoch 3/10
782/782 [==============================] - 69s 88ms/step - loss: 0.5369 - accuracy: 0.7324 - val_loss: 0.5211 - val_accuracy: 0.7333
Epoch 4/10
782/782 [==============================] - 69s 88ms/step - loss: 0.4779 - accuracy: 0.7718 - val_loss: 0.4874 - val_accuracy: 0.7679
Epoch 5/10
782/782 [==============================] - 69s 88ms/step - loss: 0.4409 - accuracy: 0.7945 - val_loss: 0.4604 - val_accuracy: 0.7778
Epoch 6/10
782/782 [==============================] - 69s 89ms/step - loss: 0.4142 - accuracy: 0.8118 - val_loss: 0.4435 - val_accuracy: 0.7941
Epoch 7/10
782/782 [==============================] - 71s 91ms/step - loss: 0.3927 - accuracy: 0.8256 - val_loss: 0.4292 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3375 - accuracy: 0.8538 - val_loss: 0.3914 - val_accuracy: 0.8229
Epoch 2/10
782/782 [==============================] - 70s 90ms/step - loss: 0.3287 - accuracy: 0.8593 - val_loss: 0.3842 - val_accuracy: 0.8258
Epoch 3/10
782/782 [==============================] - 68s 87ms/step - loss: 0.3215 - accuracy: 0.8642 - val_loss: 0.3786 - val_accuracy: 0.8315
Epoch 4/10
782/782 [==============================] - 68s 87ms/step - loss: 0.3149 - accuracy: 0.8660 - val_loss: 0.3760 - val_accuracy: 0.8325
Epoch 5/10
782/782 [==============================] - 68s 87ms/step - loss: 0.3092 - accuracy: 0.8705 - val_loss: 0.3710 - val_accuracy: 0.8338
Epoch 6/10
782/782 [==============================] - 67s 85ms/step - loss: 0.3043 - accuracy: 0.8724 - val_loss: 0.3769 - val_accuracy: 0.8316
Epoch 7/10
782/782 [==============================] - 67s 85ms/step - loss: 0.3003 - accuracy: 0.8741 - val_loss: 0.3732 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation
